# PR0501. Ingesta de datos de ficheros CSV

## Dataset 1: Datos para la predicción del rendimiento en cultivos

In [1]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
schema = StructType([
    StructField("Crop", StringType(), False),
    StructField("Region", StringType(), False),
    StructField("Soil_Type", StringType(), False),
    StructField("Soil_pH", DoubleType(), False),
    StructField("Rainfall_mm", DoubleType(), False),
    StructField("Temperature_C", DoubleType(), False),
    StructField("Humidity_pct", DoubleType(), False),
    StructField("Fertilizar_Used_kg", DoubleType(), False),
    StructField("Irrigation", StringType(), False),
    StructField("Pesticides_Used_kg", DoubleType(), False),
    StructField("Planting_Density", DoubleType(), False),
    StructField("Previous_Crop", StringType(), False),
    StructField("Yield_ton_per_ha", DoubleType(), False),
])
schema.fields

[StructField('Crop', StringType(), False),
 StructField('Region', StringType(), False),
 StructField('Soil_Type', StringType(), False),
 StructField('Soil_pH', DoubleType(), False),
 StructField('Rainfall_mm', DoubleType(), False),
 StructField('Temperature_C', DoubleType(), False),
 StructField('Humidity_pct', DoubleType(), False),
 StructField('Fertilizar_Used_kg', DoubleType(), False),
 StructField('Irrigation', StringType(), False),
 StructField('Pesticides_Used_kg', DoubleType(), False),
 StructField('Planting_Density', DoubleType(), False),
 StructField('Previous_Crop', StringType(), False),
 StructField('Yield_ton_per_ha', DoubleType(), False)]

In [3]:
from pyspark.sql import SparkSession
spark = ( SparkSession.builder
            .appName("pruebas")
            .master("spark://spark-master:7077")
            .getOrCreate()
        )
df = (
    spark.read
        .format("csv")
        .schema(schema)
        .option("header", "true")
        .load("/workspace/pr0501/crop_yield_dataset.csv")
)

### 1.-Selección de características

In [4]:
df_sel = df.select("Crop", "Region", "Temperature_C", "Rainfall_mm", "Irrigation", "Yield_ton_per_ha")
df_sel

DataFrame[Crop: string, Region: string, Temperature_C: double, Rainfall_mm: double, Irrigation: string, Yield_ton_per_ha: double]

### 2.-Normalización de los nombres

In [6]:
df_renamed = df_sel\
    .withColumnRenamed("Temperature_C", "Temperatura") \
    .withColumnRenamed("Rainfall_mm", "Lluvia") \
    .withColumnRenamed("Yield_ton_per_ha", "Rendimiento")
df_renamed

DataFrame[Crop: string, Region: string, Temperatura: double, Lluvia: double, Irrigation: string, Rendimiento: double]

### 3.-Filtrado de datos (`filter`)

In [9]:
from pyspark.sql.functions import col, lit
df_filtered = df_renamed.filter((col("Crop") == "Maize") & (col("Temperatura") > 25))
df_filtered.show(3)

+-----+--------+-----------+------+----------+-----------+
| Crop|  Region|Temperatura|Lluvia|Irrigation|Rendimiento|
+-----+--------+-----------+------+----------+-----------+
|Maize|Region_D|       26.4|1054.3|      Drip|     169.06|
|Maize|Region_C|       32.4| 846.1|      None|      162.2|
|Maize|Region_A|       26.6| 362.5| Sprinkler|      95.23|
+-----+--------+-----------+------+----------+-----------+
only showing top 3 rows



### 4.-Encadenamiento

In [10]:
df_all_in_one = df \
    .select("Crop", "Region", "Temperature_C", "Rainfall_mm", "Irrigation", "Yield_ton_per_ha") \
    .withColumnRenamed("Temperature_C", "Temperatura") \
    .withColumnRenamed("Rainfall_mm", "Lluvia") \
    .withColumnRenamed("Yield_ton_per_ha", "Rendimiento") \
    .filter((col("Crop") == "Maize") & (col("Temperatura") > 25))
df_all_in_one.show(5)


+-----+--------+-----------+------+----------+-----------+
| Crop|  Region|Temperatura|Lluvia|Irrigation|Rendimiento|
+-----+--------+-----------+------+----------+-----------+
|Maize|Region_D|       26.4|1054.3|      Drip|     169.06|
|Maize|Region_C|       32.4| 846.1|      None|      162.2|
|Maize|Region_A|       26.6| 362.5| Sprinkler|      95.23|
|Maize|Region_C|       33.7|1193.3|      None|     110.57|
|Maize|Region_C|       27.8| 695.2|     Flood|     143.84|
+-----+--------+-----------+------+----------+-----------+
only showing top 5 rows



## Dataset 2: Lugares famosos del mundo

In [12]:
from pyspark.sql.types import IntegerType
schema = StructType([
    StructField("Place_Name", StringType(), False),
    StructField("Country", StringType(), False),
    StructField("City", StringType(), False),
    StructField("Annual_Visitors_Millions", IntegerType(), False),
    StructField("Type", StringType(), False),
    StructField("UNESCO_World_Heritage", StringType(), False),
    StructField("Year_Build", StringType(), False),
    StructField("Entry_Fee_USD", IntegerType(), False),
    StructField("Best_Visit_Month", StringType(), False),
    StructField("Region", StringType(), False),
    StructField("Tourism_Revenue_Million_USD", IntegerType(), False),
    StructField("Average_Visit_Duration_Hours", DoubleType(), False),
    StructField("Famous_For", StringType(), False),
])

schema.fields

[StructField('Place_Name', StringType(), False),
 StructField('Country', StringType(), False),
 StructField('City', StringType(), False),
 StructField('Annual_Visitors_Millions', IntegerType(), False),
 StructField('Type', StringType(), False),
 StructField('UNESCO_World_Heritage', StringType(), False),
 StructField('Year_Build', StringType(), False),
 StructField('Entry_Fee_USD', IntegerType(), False),
 StructField('Best_Visit_Month', StringType(), False),
 StructField('Region', StringType(), False),
 StructField('Tourism_Revenue_Million_USD', IntegerType(), False),
 StructField('Average_Visit_Duration_Hours', DoubleType(), False),
 StructField('Famous_For', StringType(), False)]

In [13]:
from pyspark.sql import SparkSession
spark = ( SparkSession.builder
            .appName("pruebas")
            .master("spark://spark-master:7077")
            .getOrCreate()
        )
df = (
    spark.read
        .format("csv")
        .schema(schema)
        .option("header", "true")
        .load("/workspace/pr0501/world_famous_places_2024.csv")
)

#### 1.-Selección de datos críticos

In [15]:
df_base = df.select("Place_Name", "Country", "UNESCO_World_Heritage", "Entry_Fee_USD", "Annual_Visitors_Millions")
df_base

DataFrame[Place_Name: string, Country: string, UNESCO_World_Heritage: string, Entry_Fee_USD: int, Annual_Visitors_Millions: int]

### 2.-Traducción y simplificación

In [16]:
df_es = df_base \
    .withColumnRenamed("Place_Name", "Lugar") \
    .withColumnRenamed("UNESCO_World_Heritage", "Es_UNESCO") \
    .withColumnRenamed("Entry_Fee_USD", "Precio_Entrada") \
    .withColumnRenamed("Annual_Visitors_Millions", "Visitantes_Millones")
df_es

DataFrame[Lugar: string, Country: string, Es_UNESCO: string, Precio_Entrada: int, Visitantes_Millones: int]

## Dataset 3: Registro turístico de Castilla y León

In [7]:
schema = StructType([
    StructField("establecimiento", StringType(), False),
    StructField("n_registro", StringType(), False),
    StructField("codigo", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("categoria", StringType(), True),
    StructField("especialidades", StringType(), True),
    StructField("clase", StringType(), True),
    StructField("nombre", StringType(), False),
    StructField("direccion", StringType(), False),
    StructField("c_postal", StringType(), False),
    StructField("porvincia", StringType(), False),
    StructField("municipio", StringType(), True),
    StructField("localidad", StringType(), True),
    StructField("nucleo", StringType(), True),
    StructField("telefono_1", StringType(), True),
    StructField("telefono_2", StringType(), True),
    StructField("telefono_3", StringType(), True),
    StructField("email", StringType(), True),
    StructField("web", StringType(), True),
    StructField("q_calidad", StringType(), True),
    StructField("posada_real", StringType(), True),
    StructField("plazas", StringType(), True),
    StructField("gps_longitud", DoubleType(), True),
    StructField("gps_latitud", DoubleType(), True),
    StructField("accesible_a_personas_con_discapacidad", StringType(), True),
    StructField("column_27", StringType(), True),
    StructField("posicion", StringType(), True),
])

In [ ]:
df = (
    spark.read
        .format("csv")
        .schema(schema)
        .option("delimiter", ";")
        .option("header", "true")
        .option("quote", "")
        .option("escape", "\\")
        .option("multiLine", "true")   
        .option("emptyValue", None)    
        .option("nullValue", "NULL")
        .load("/workspace/pr0501/registro-de-turismo-de-castilla-y-leon.csv")
)

In [9]:
df.printSchema()
df.show(5)

root
 |-- establecimiento: string (nullable = true)
 |-- n_registro: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- especialidades: string (nullable = true)
 |-- clase: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- c_postal: string (nullable = true)
 |-- porvincia: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- localidad: string (nullable = true)
 |-- nucleo: string (nullable = true)
 |-- telefono_1: string (nullable = true)
 |-- telefono_2: string (nullable = true)
 |-- telefono_3: string (nullable = true)
 |-- email: string (nullable = true)
 |-- web: string (nullable = true)
 |-- q_calidad: string (nullable = true)
 |-- posada_real: string (nullable = true)
 |-- plazas: string (nullable = true)
 |-- gps_longitud: double (nullable = true)
 |-- gps_latitud: double (nullable = true)
 |-- accesible_a_pers

26/01/15 11:00:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----------+------+--------------------+---------------+--------------+-----+--------------------+--------------------+--------+---------+---------+---------------+---------------+----------+----------+----------+--------------------+--------------------+---------+-----------+------+------------+-----------+-------------------------------------+---------+--------------------+
|     establecimiento|n_registro|codigo|                tipo|      categoria|especialidades|clase|              nombre|           direccion|c_postal|porvincia|municipio|      localidad|         nucleo|telefono_1|telefono_2|telefono_3|               email|                 web|q_calidad|posada_real|plazas|gps_longitud|gps_latitud|accesible_a_personas_con_discapacidad|column_27|            posicion|
+--------------------+----------+------+--------------------+---------------+--------------+-----+--------------------+--------------------+--------+---------+---------+---------------+---------------

26/01/15 11:00:40 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
